# Useful decorators

## classmethod / staticmethod

**@classmethod** заставляет декорируемую функцию оперировать самим классом, а не конкретным его инстансом (не требует создания объекта-инстанса класса) 

**@staticmethod** "отвязывает" декорируемую функцию от ее класса в том смысле, что функция теряет доступ к полям класса и полям инстанса класса. Но зато и не требует создания инстанса для своей работы

In [1]:
class MethodsDemo:
    NAME = 'methods_demo'
    
    def __init__(self, nickname='default', *args):
        self.nickname = nickname
        
    def do(self, *args):
        return args
    
    def access_fields(self, *args):
        return self.NAME, self.nickname
    
    @classmethod
    def do_classmethod(*args):
        return args
    
    @classmethod
    def access_fields_cls(cls, *args):
        return cls.NAME
    
    @classmethod
    def generate_cls(cls, *args):
        return cls(*args)
    
    @staticmethod
    def do_staticmethod(*args):
        return args

In [2]:
dir(MethodsDemo)

['NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'access_fields',
 'access_fields_cls',
 'do',
 'do_classmethod',
 'do_staticmethod',
 'generate_cls']

### Вызовы

In [3]:
MethodsDemo.do_classmethod('param')

(__main__.MethodsDemo, 'param')

In [4]:
MethodsDemo.do_staticmethod('param')

('param',)

In [5]:
MethodsDemo.do('param')

()

### Доступы к полям

In [6]:
MethodsDemo.access_fields()

TypeError: access_fields() missing 1 required positional argument: 'self'

In [ ]:
MethodsDemo().access_fields()

In [ ]:
MethodsDemo.access_fields_cls()

staticmethod не имеет доступов к полям, classmethod имеет доступ только к полям класса

In [ ]:
MethodsDemo.generate_cls('new_nick').access_fields()

In [29]:
MethodsDemo.__dict__

mappingproxy({'__module__': '__main__',
              'NAME': 'methods_demo',
              '__init__': <function __main__.MethodsDemo.__init__(self, nickname='default', *args)>,
              'do': <function __main__.MethodsDemo.do(self, *args)>,
              'access_fields': <function __main__.MethodsDemo.access_fields(self, *args)>,
              'do_classmethod': <classmethod at 0x7f4793f38250>,
              'access_fields_cls': <classmethod at 0x7f4793f38990>,
              'generate_cls': <classmethod at 0x7f4793f386d0>,
              'do_staticmethod': <staticmethod at 0x7f4793f38e10>,
              '__dict__': <attribute '__dict__' of 'MethodsDemo' objects>,
              '__weakref__': <attribute '__weakref__' of 'MethodsDemo' objects>,
              '__doc__': None})

In [30]:
MethodsDemo().__dict__

{'nickname': 'default'}

## property

In [7]:
class PropertyShow:
    def __init__(self, var):
        self._var = var
    
    @property
    def x(self):
        return self._var

In [8]:
cls = PropertyShow(42)

In [9]:
cls.x

42

In [10]:
cls._var

42

In [11]:
class NotPropertyShow:
    def __init__(self, var):
        self._var = var
    
    def x(self):
        return self._var

In [12]:
cls = NotPropertyShow(42)

In [13]:
cls.x()

42

In [14]:
cls._var

42

To understand what's going on, read great SO answer [here](https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python)

## class as a decorator

Экземпляры класса могут быть callable. А если функция может быть декоратором, то что мешает тогда классу? Должен быть определен magic метод `__call__`

In [54]:
class ClassDecorator:
    def __init__(self, func):
        self.entry_str = '[decorating inner func]'
        self.function = func
        self.close_str = '[letting the song flow]'

     
    def __call__(self):
        print(self.entry_str)
        self.function()
        print(self.close_str)

 
 
@ClassDecorator
def function():
    print('Vanderlyle crybaby cry...')

In [55]:
function()

[decorating inner func]
Vanderlyle crybaby cry...
[letting the song flow]


Как и в декораторах-функциях, мы тоже можем возвращать значения (а не только использовать print) и делать параметры

Сделаем таймер

In [ ]:
import time

class Timer:
    pass

In [64]:
@Timer
def slow_func(seconds_to_sleep):
    pass

In [66]:
slow_func(3)

slow_func((3,)) -> 42 executed in 3.00s


42

## class as decorated

Класс может быть тоже декорирован функцией или классом

In [193]:
def decorator_function(target):

    def decorator_init(self, *args):
        print("Decorator running")
        print(*args)
    target.__init__ = decorator_init
    return target


class Target:
    def __init__(self, *args):
        self.arg_list = args
        print("Target running")


t = Target('one', 1)
t.__dict__

Target running


{'arg_list': ('one', 1)}

In [194]:
@decorator_function
class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")

dt = Target('one', 1)
dt.__dict__

Decorator running
one 1


{}

# Descriptors

Короткое описание в [доке](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)

Мы самом деле, мы с вами уже выше познакомились с частным примером такой сущности.

Дескриптор -- это объект с определенными методами `__get__()`, `__set__()` и `__delete__()`

Обычно когда в Python мы хотим получить доступ к атрибуту класса, то сначала мы ищем его в `__dict__` инстанса класса, потом в классе, и далее выше по цепочке наследования.

## Basics

In [106]:
class BestConstant:
    def __get__(self, obj, objtype=None):
        print('Using descriptor')
        return 73

class Number:
    x = 10
    y = BestConstant()  # наш дескриптор

In [107]:
num = Number()

In [108]:
num.x, num.y

Using descriptor


(10, 73)

In [109]:
num.__dict__, Number.__dict__

({},
 mappingproxy({'__module__': '__main__',
               'x': 10,
               'y': <__main__.BestConstant at 0x7f4793ec2b50>,
               '__dict__': <attribute '__dict__' of 'Number' objects>,
               '__weakref__': <attribute '__weakref__' of 'Number' objects>,
               '__doc__': None}))

In [110]:
num.__dict__['y'] = 42

In [111]:
num.y

42

Видим, что мы переписали значение. Дескриптор, у которого определен только get, называется non-data дескриптором

In [98]:
class BestConstantComplete:
    def __get__(self, obj, objtype=None):
        print('using descriptor')
        return obj._y
    
    def __set__(self, obj, value):
        obj._y = value
        
    def __delete__(self, obj):
        del obj._y

class Number:
    y = BestConstantComplete()  # наш дескриптор
    
    def __init__(self, x=10, y=42):
        self.x = x
        self.y = y

In [99]:
num = Number()

In [100]:
num.y

using descriptor


42

In [101]:
num.y = 73

In [102]:
num.y

using descriptor


73

In [103]:
num.__dict__

{'x': 10, '_y': 73}

In [104]:
num.__dict__['y'] = 100

In [105]:
num.y

using descriptor


73

Если у дескриптора определен `__set__`, то Python при попытке достать атрибут по названию, будет доставать сначала дескриптор, даже если в `__dict__` объекта лежит что-то одноименное 

## Closer to Real world

Например, с помощью дескрипторов можно написать логгер запросов к полю класса.

Пример из документации:

In [113]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info(f'Accessing "age" giving {value}')
        return value

    def __set__(self, obj, value):
        logging.info(f'Updating age to {value}')
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1 

In [114]:
denis = Person('Denis Belyakov', 23)

INFO:root:Updating age to 23


In [115]:
vars(denis)

{'name': 'Denis Belyakov', '_age': 23}

In [116]:
denis.__dict__

{'name': 'Denis Belyakov', '_age': 23}

In [117]:
denis.age

INFO:root:Accessing "age" giving 23


23

In [118]:
denis.name

'Denis Belyakov'

In [119]:
denis._age

23

## And yes, property

In [121]:
property.__dict__

mappingproxy({'__getattribute__': <slot wrapper '__getattribute__' of 'property' objects>,
              '__get__': <slot wrapper '__get__' of 'property' objects>,
              '__set__': <slot wrapper '__set__' of 'property' objects>,
              '__delete__': <slot wrapper '__delete__' of 'property' objects>,
              '__init__': <slot wrapper '__init__' of 'property' objects>,
              '__new__': <function property.__new__(*args, **kwargs)>,
              'getter': <method 'getter' of 'property' objects>,
              'setter': <method 'setter' of 'property' objects>,
              'deleter': <method 'deleter' of 'property' objects>,
              'fget': <member 'fget' of 'property' objects>,
              'fset': <member 'fset' of 'property' objects>,
              'fdel': <member 'fdel' of 'property' objects>,
              '__doc__': <member '__doc__' of 'property' objects>,
              '__isabstractmethod__': <attribute '__isabstractmethod__' of 'property' objec

    The main motivation for descriptors is to provide a hook allowing objects stored in class variables to control what happens during attribute lookup.

Хорошая [лекция](http://uneex.org/LecturesCMC/PythonIntro2020/11_MiscOOP) на тему

Развернутый [док](https://docs.python.org/3/howto/descriptor.html)

In [15]:
@check_annotation
def foo(x: int):
    return x + 1

NameError: name 'check_annotation' is not defined

# Inheritance

## Basics

Синтаксис простой

In [122]:
class BetterList(list):
    # поля и методы
    pass

Порядок видимости снизу вверх: объект -> класс -> родительский класс

Для доступа к полям родителя используется `super()`

In [129]:
class Parent:
    def letter(self):
        return 'D'
    
class Child(Parent):
    def letter(self):
        return super().letter() + 'B'

In [130]:
a = Parent()
b = Child()

In [131]:
a.letter()

'D'

In [132]:
b.letter()

'DB'

In [146]:
class Parent:
    def __init__(self, letter='D'):
        self._letter = letter
        
    def get_letter(self):
        return self._letter
    
class Child(Parent):
    def __init__(self, parent_letter='D', child_letter='B'):
        super().__init__(parent_letter)
        print(f'parent inited {self._letter}')
        self.new_letter = child_letter
        
    def get_child_letter(self):
        return self.new_letter

In [147]:
a = Child()

parent inited D


In [148]:
a.get_letter()

'D'

In [149]:
a.get_child_letter()

'B'

In [150]:
a.__dict__

{'_letter': 'D', 'new_letter': 'B'}

## Protecting names

Фичи наследования в доопределении пространства имен и в возможности их перегрузки  

Предполагается, что если пользователь переопредляет имя, значит он этого хочет 

Если создатель класса не хочет, чтобы какое-то поле было явно перегружено, то можно, например, использовать дескрипторы или особый нейминг

Есть общее соглашение, что переменные, названные с `_` не надо трогать вне класса, в котором они объявлены

Также в Python предусмотрен механизм с названием, начинающися с `__`

In [155]:
class StrictParent:
    def __init__(self):
        self.__letter = 'D'

class Child(StrictParent):
    def __init__(self):
        super().__init__()
        self._letter = 'B'

In [156]:
a = Child()

In [157]:
a.__dict__

{'_StrictParent__letter': 'D', '_letter': 'B'}

# Exceptions

Оператор **try-except-else-finally**

In [163]:
a = '123'

try:
    a[0] = '10'
except Exception:
    print('impossible to modify object')

impossible to modify object


In [160]:
a[0] = '10'

TypeError: 'str' object does not support item assignment

In [162]:
TypeError.mro()

[TypeError, Exception, BaseException, object]

In [166]:
a = [1, 2, 3]

try:
    a[0] = '10'
except Exception:
    print('impossible to modify object')
else:
    print('possible to modify object', a)

possible to modify object ['10', 2, 3]


Исключения — не «ошибки», а способ обработки некоторых условий не там, где они были обнаружены.

In [167]:
raise TypeError

TypeError: 

In [168]:
raise TypeError('message')

TypeError: message

Хорошая [лекция](http://uneex.org/LecturesCMC/PythonIntro2020/10_Inheritance) по механизму наследования в т.ч. с примерами исключений 

**Bonus:**

Напишите декоратор-класс, который бы:
1. Проверял входные аргументы и вывод функции на соответствие типам, указанным в аннотациях 
2. Проверял, что все численные типы находятся в диапазоне от -1000 до 1000
3. Длины типов-последовательностей не превосходят 1000
4. В случае ошибок валидации возвращал информативное сообщение об ошибке в виде Exception
5. Мог бы быть отключаемым по параметрам

Для решения используйте дескрипторы, вам очень помогут упомянутые сегодня ссылки 

In [239]:
def switch_off(switch=True):
    def decorator(target):
        
        def decorator_init(self, func):
            self.func = func
            self._annotations = func.__annotations__
            self.switch = switch

        if switch:
            target.__init__ = decorator_init

        return target
    return decorator

In [240]:
@switch_off(switch=True)
class CheckType:
    def __init__(self, func):
        self.func = func
        self._annotations = func.__annotations__
        self.switch = False

    def __call__(self, *args):
        if self.switch:
            for var, arg in zip(self._annotations, args):
                if not isinstance(arg, self._annotations[var]):
                    raise Exception(f"{var} doesn't match {self._annotations[var]}")
                else:
                    if isinstance(arg, int) and (-1000 > arg or arg > 1000):
                        raise Exception(f"{var} can only be in range (-1000, 1000)")
                    elif isinstance(arg, list) and len(arg) > 1000:
                        raise Exception(f"Length of the list {var} has to be less that 1000")

            res = self.func(*args)
            if not isinstance(res, self._annotations['return']):
                raise Exception(f"Output doesn't match {self._annotations['return']}")
        else:
            res = self.func(*args)
        return res

In [241]:
@CheckType
def increase_list(a : list, n : int) -> list:
    for i in range(len(a)):
        a[i] += n
    return a[i]

In [243]:
increase_list("sdf", 2)

Exception: a doesn't match <class 'list'>

In [244]:
increase_list([1, 2], [1, 2])

Exception: n doesn't match <class 'int'>

In [245]:
increase_list(list(range(10000)), 2)

Exception: Length of the list a has to be less that 1000

In [246]:
increase_list([1, 2], -100000)

Exception: n can only be in range (-1000, 1000)

In [247]:
@CheckType
def increase_list(a : list, n : int) -> list:
    for i in range(len(a)):
        a[i] += n
    return n

In [249]:
increase_list([1, 2], 2)

Exception: Output doesn't match <class 'list'>